In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.unicode_minus'] = False  


xls_new = pd.ExcelFile(r'NDVI_all.xlsx')


dfs_new = [xls_new.parse(x) for x in xls_new.sheet_names]


def check_recovery(row):
    recovery_status = 2
    year_value = None
      
    

    for i in range(len(row)):
        if (row[i:i+2] > 0).all():
            recovery_status = 1
            year_value = i+1
            break

    if recovery_status == 2:

        if (row[-4:] < 0).all() and (row[-4:].diff().sum() < 0):
            recovery_status = 3
            year_value = 999
        if recovery_status == 2:
            year_value = 999  

    return recovery_status, year_value


for i, df in enumerate(dfs_new, start=1):
    dNBR=df['dNBR']


    df = df.iloc[:, -8:].sub(df.iloc[:, -9], axis=0) 
    

    df[['Recovery', 'Year']] = df.apply(check_recovery, axis=1, result_type='expand').astype(int)
    
    df['dNBR']=dNBR


    recovery_1 = df[df['Recovery'] == 1]
    recovery_2 = df[df['Recovery'] == 2]
    recovery_3 = df[df['Recovery'] == 3]


    df_filtered = pd.concat([recovery_1, recovery_2, recovery_3])

    df_filtered['Recovery'] = df_filtered['Recovery'].astype(str)


    stat, p_value = stats.kruskal(recovery_1['dNBR'], recovery_2['dNBR'], recovery_3['dNBR'])
    print(f"Dataset {i} - Kruskal-Wallis H-test: H-statistic = {stat:.3f}, p-value = {p_value:.5f}")



    palette = {'1': '#1F992B', '2': '#6FFF7E', '3': '#FC4545'}


    plt.figure(figsize=(5, 4))
    sns.boxplot(x='Recovery', y='dNBR', data=df_filtered, showmeans=True, 
                meanprops={"marker": "o", "markerfacecolor": "red", "markeredgecolor": "black"},
                palette=palette)
    
    plt.ylim(0,0.7)
    

    plt.xticks([0, 1, 2], ['Recovered', 'Recovering', 'Recovery hysteresis'], rotation=10,fontsize=15, weight='normal')
    plt.yticks(fontproperties='Arial', fontsize=15, weight='normal')

    plt.ylabel('Mean dNBR',fontsize=18, weight='semibold')
    plt.xlabel('Category',fontsize=18, weight='semibold')
    
#     plt.savefig(f"BoxdNBR_GPP_{i}.jpg",bbox_inches = 'tight',dpi=600)    
    
    
    plt.show()